In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models
import cv2
import os
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping , ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import Input
print('DONE')


DONE


In [3]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 14.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install mmaction

  Using cached mmaction-0.5.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached mmcv-2.2.0.tar.gz (479 kB)
  Preparing metadata (setup.py) ... done
  Using cached Pillow-6.2.2-cp310-cp310-linux_x86_64.whl
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached mmengine-0.10.5-py3-none-any.whl.metadata (20 kB)
Using cached mmaction-0.5.0-py2.py3-none-any.whl (106 kB)
Using cached mmengine-0.10.5-py3-none-any.whl (452 kB)
Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
  Created wheel for mmcv: filename=mmcv-2.2.0-cp310-cp310-linux_x86_64.whl size=1366539 sha256=d272502fb07f6ff9bd9f3d6b62d6cffadff8afe1ebe44e8566cb999b4ada3b56
  Stored in directory: /root/.cache/pip/wheels/dd/50/9f/4e0c45c952bddd25de1cae1b310bd1b20dd69967b5202ca1af
Successfully built mmcv
  Attempting uninstall: Pillow
    Found existing installation: pillow 10.3.0
    Uninstalling pillow-10.3.0:
      Successfully uninstalled pillow-10.3.0
ERROR: pip's dependency resolver does not currently 

In [11]:
pip install mmcv

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install mmaction2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 13.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 82.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from pathlib import Path
import decord
from mmengine import Config
from mmaction.apis import init_recognizer
from mmaction.registry import DATASETS, TRANSFORMS
from functools import partial
from multiprocessing import Manager, cpu_count
print('DONE')



DONE


In [36]:
def init_tsn_model():
    config_file = '/kaggle/input/mmaction/mmaction2-main/configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-dense-1x1x5-100e_kinetics400-rgb.py'
    checkpoint_file = '/kaggle/input/mmaction/mmaction2-main/tools/analysis_tools/check_videos.py'
    
    cfg = Config.fromfile(config_file)
    model = init_recognizer(cfg, checkpoint_file, device='cuda')
    model.eval()
    return model

def preprocess_frame_for_tsn(frame):
    h, w = frame.shape[:2]
    new_h = 256
    new_w = int(w * (new_h / h))
    frame = cv2.resize(frame, (new_w, new_h))
    
    start_x = (new_w - 224) // 2
    start_y = (new_h - 224) // 2
    frame = frame[start_y:start_y+224, start_x:start_x+224]
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = frame / 255.0
    frame = (frame - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])
    frame = frame.transpose(2, 0, 1)
    return frame
print('DONE')

DONE


In [37]:
def process_frame_multistream(frame, yolo_model, tsn_model):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    yolo_results = yolo_model(frame_rgb)
    
    frame_processed = preprocess_frame_for_tsn(frame)
    frame_tensor = torch.from_numpy(frame_processed).unsqueeze(0).float().cuda()
    
    with torch.no_grad():
        full_frame_features = tsn_model.extract_feat(frame_tensor)
        if isinstance(full_frame_features, tuple):
            full_frame_features = full_frame_features[0]
    
    clipped_features = []
    yolo_positions = []
    
    if len(yolo_results) > 0 and yolo_results[0].boxes:
        for box in yolo_results[0].boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            h, w = frame.shape[:2]
            
            padding = 20
            x1, y1 = max(0, x1 - padding), max(0, y1 - padding)
            x2, y2 = min(w, x2 + padding), min(h, y2 + padding)
            
            clipped_frame = frame[y1:y2, x1:x2]
            if clipped_frame.size > 0:
                processed_clip = preprocess_frame_for_tsn(clipped_frame)
                clip_tensor = torch.from_numpy(processed_clip).unsqueeze(0).float().cuda()
                
                with torch.no_grad():
                    clip_features = tsn_model.extract_feat(clip_tensor)
                    if isinstance(clip_features, tuple):
                        clip_features = clip_features[0]
                
                clipped_features.append(clip_features.cpu().numpy())
                yolo_positions.append([x1/w, y1/h, x2/w, y2/h])
    
    return (
        full_frame_features.cpu().numpy(),
        np.array(yolo_positions) if yolo_positions else np.zeros((0, 4)),
        np.array(clipped_features) if clipped_features else np.zeros((0, full_frame_features.shape[1]))
    )
print('DONE')


DONE


In [39]:
def process_video_multistream(video_path, yolo_model, tsn_model, frame_interval=5):
    try:
        vr = decord.VideoReader(video_path)
        total_frames = len(vr)
        
        full_frame_features = []
        all_yolo_positions = []
        all_clipped_features = []
        
        frame_indices = range(0, total_frames, frame_interval)
        for frame_idx in tqdm(frame_indices, desc=f"Processing {Path(video_path).name}"):
            frame = vr[frame_idx].asnumpy()
            frame_feats, positions, clipped_feats = process_frame_multistream(
                frame, yolo_model, tsn_model
            )
            
            full_frame_features.append(frame_feats)
            all_yolo_positions.append(positions)
            all_clipped_features.append(clipped_feats)
        
        return {
            'full_frame_features': np.array(full_frame_features),
            'yolo_positions': all_yolo_positions,
            'clipped_features': all_clipped_features,
            'frame_indices': np.array(frame_indices),
            'total_frames': total_frames
        }
    except Exception as e:
        print(f"Error processing video {video_path}: {str(e)}")
        return None
print('DONE')

DONE


In [42]:
def process_dataset(input_dirs, train_output_path, val_output_path, yolo_model, val_ratio=0.2):
    tsn_model = init_tsn_model()
    
    os.makedirs(train_output_path, exist_ok=True)
    os.makedirs(val_output_path, exist_ok=True)
    
    video_paths = []
    for input_dir in input_dirs:
        video_paths.extend([
            os.path.join(input_dir, f) 
            for f in os.listdir(input_dir) 
            if f.lower().endswith(('.mp4', '.avi', '.mov'))
        ])
    
    random.shuffle(video_paths)
    split_idx = int(len(video_paths) * (1 - val_ratio))
    train_videos = video_paths[:split_idx]
    val_videos = video_paths[split_idx:]
    
    for videos, output_path in [(train_videos, train_output_path), 
                              (val_videos, val_output_path)]:
        for video_path in tqdm(videos):
            features = process_video_multistream(video_path, yolo_model, tsn_model)
            if features is not None:
                video_name = Path(video_path).stem
                save_path = os.path.join(output_path, f"{video_name}_multistream.npz")
                np.savez_compressed(save_path, **features)
print('DONE')




DONE


In [45]:
if __name__ == "__main__":
    # Set up output directories
    output_paths = setup_output_paths("/kaggle/working/processed_features")
    
    # Process violence dataset
    process_videos(
        ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence"],
        output_paths["train_violence"],
        output_paths["val_violence"],
        split_for_val=True
    )
    
    # Process non-violence dataset
    process_videos(
        ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence"],
        output_paths["train_nonviolence"],
        output_paths["val_nonviolence"],
        split_for_val=True
    )

Processing 800 training videos...


Processing training videos: 100%|██████████| 800/800 [00:00<00:00, 94947.46it/s]


Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_211.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_82.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_545.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_233.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_262.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_25.mp4: name 'process_video' is not defined


Processing validation videos: 100%|██████████| 200/200 [00:00<00:00, 105239.09it/s]


Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_310.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_686.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_707.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_814.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_457.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_389.mp4: name 'process_video' i

Processing training videos: 100%|██████████| 800/800 [00:00<00:00, 112001.17it/s]


Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_150.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_450.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_782.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_857.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_639.mp4: name 'process_video' is not defined
Error processing training video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_211.mp4: name 'proc

Processing validation videos: 100%|██████████| 200/200 [00:00<00:00, 71065.81it/s]

Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_757.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_577.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_156.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_521.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_445.mp4: name 'process_video' is not defined
Error processing validation video /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_31.mp4: